Run this notebook in the same virtual environment with superlinked server
to ensure the same version of `superlinked` framework.

In [1]:
!pip freeze | grep superlinked

superlinked==27.0.0
superlinked-server==1.27.0


In [2]:
from pathlib import Path
import sys

# depending on the user's setup
# we will try to find the superlinked_app directory
# and add it to the sys.path

cwd = Path.cwd()
if cwd.name == "superlinked-recipes":
    project_dir = cwd / "projects" / "recipe-search"
elif cwd.name == "notebooks":
    project_dir = cwd.parent
else:
    project_dir = cwd

superlinked_app_dir = project_dir / "superlinked_app"
assert superlinked_app_dir.exists(), (
    f"{superlinked_app_dir} does not exist\n"
    "are you sure you are in the recipe-search/notebooks directory?"
)

if str(project_dir) not in sys.path:
    sys.path.append(str(project_dir))
    print("project_dir is added to sys.path")
else:
    print("project_dir is already in sys.path")

project_dir is added to sys.path


In [3]:
print(project_dir)

/home/biso/development/my_projects/varie/sherlockrecipes


In [4]:
from superlinked import framework as sl

from superlinked_app.index import index, recipe_schema
from superlinked_app.query import query
from superlinked_app.api import vector_database

import pandas as pd

19:04:36 superlinked.framework.dsl.index.index INFO   initialized index


In [5]:
source = sl.InteractiveSource(recipe_schema)
executor = sl.InteractiveExecutor(
    sources=[source],
    indices=[index],
    vector_database=vector_database,
)
app = executor.run()

19:04:36 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
19:04:36 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/exists "HTTP/1.1 200 OK"
19:04:36 httpx INFO   HTTP Request: GET https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default "HTTP/1.1 200 OK"
19:04:37 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/index?wait=true "HTTP/1.1 200 OK"
19:04:37 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/index?wait=true "HTTP/1.1 200 OK"
19:04:37 httpx INFO   HTTP Request: PUT https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/index?wait=true "HTTP/1.1 200 OK"
19:04

In [6]:
# params = {
#     "natural_query": "well rated pet friendly hotel in london, cozy interior",
#     "limit": 3,
# }

# result = app.query(query, **params)
# print(result.metadata.search_params)
# sl.PandasConverter.to_pandas(result)

In [41]:
# Example manual query parameters
params = {
    "name_query": "Pizza",              # Text to search in recipe names
    "ingredients_query": "greek_yogurt", # Text to search in ingredients
    "instructions_query": "Preheat",      # Text to search in instructions
    "limit": 5,                                  # Number of results to return

    # Weights for similarity spaces (adjust as needed)
    "name_weight": 1.0,
    "ingredients_weight": 1.0,
    "instructions_weight": 1.0,

    # Similarity weights (can be same as above or different)
    "similar_name_weight": 1.0,
    "similar_ingredients_weight": 1.0,
    "similar_instructions_weight": 1.0,

    # Numerical filters (set to None or appropriate values)
    "min_rating": 4.0,
    #"max_rating": 15.0,
    "min_prep_time": 0.0,
    "max_prep_time": 9000.0,
    "min_cook_time": 0.0,
    "max_cook_time": 1000.0,
    "max_calories": 400.0,

    # Categorical filters (empty list or specify categories/cuisines)
    # Remove or set to None to disable categories filtering
    #"categories_include_any": ["Dinner"],
    #"categories_exclude": [],
    # Remove or set to None to disable cuisine filtering
    "cuisines_include_any": ["Italian"],
    # "cuisines_exclude": [],
}

# Run the query with manual parameters
result = app.query(query, **params)

# Print search parameters and convert results to pandas DataFrame

# Show all rows

print(result.metadata.search_params)
df = sl.PandasConverter.to_pandas(result)
df


19:18:17 superlinked.framework.query.query_dag_evaluator INFO   evaluated query


19:18:18 httpx INFO   HTTP Request: POST https://bad13d43-afc8-44b5-8b37-2fa1eb4f0236.eu-west-1-0.aws.cloud.qdrant.io:6333/collections/default/points/query "HTTP/1.1 200 OK"
19:18:18 superlinked.framework.dsl.executor.query.query_executor INFO   executed query
{'name_query': 'Pizza', 'similar_name_weight': 1.0, 'ingredients_query': 'greek_yogurt', 'similar_ingredients_weight': 1.0, 'instructions_query': 'Preheat', 'similar_instructions_weight': 1.0, 'limit': 5, 'select_param__': ['Name', 'Ingredient_Names_Text', 'Instructions', 'Rating_Value', 'Preparation_Time', 'Cooking_Time', 'Category', 'Cuisine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fiber', 'Protein', 'Saturated_Fat', 'Sodium', 'Sugar', 'Fat', 'Unsaturated_Fat', 'Nutrition', 'Ingredients', 'URL'], 'min_rating': 4.0, 'max_rating': None, 'min_prep_time': 0.0, 'max_prep_time': 9000.0, 'min_cook_time': 0.0, 'max_cook_time': 1000.0, 'max_calories': 400.0, 'categories_include_all': None, 'categories_include_any': None, 'categori

,Name,Ingredient_Names_Text,Instructions,Rating_Value,Preparation_Time,Cooking_Time,Category,Cuisine,Calories,Carbohydrates,...,Saturated_Fat,Sodium,Sugar,Fat,Unsaturated_Fat,Nutrition,Ingredients,URL,id,similarity_score
0,Two-Ingredient Pizza Dough,self-rising_flour greek_yogurt spray,Gather all ingredients. Preheat the oven to 50...,4.6,15.0,10.0,[Dinner],[Italian],116.0,18.0,...,1.0,314.0,1.0,3.0,0.0,"{""Calories"": ""116 kcal"", ""Carbohydrates"": ""18 ...","[{'ingredient': 'self-rising flour', 'quantity...",https://www.allrecipes.com/recipe/244447/two-i...,rec136,0.492732
1,Lemon Ricotta Cake,cake_flour baking_powder baking_soda salt whit...,Gather the ingredients. Preheat the oven to 35...,4.9,20.0,40.0,[Cake],[Italian],340.0,44.0,...,9.0,336.0,27.0,16.0,0.0,"{""Calories"": ""340 kcal"", ""Carbohydrates"": ""44 ...","[{'ingredient': 'cake flour', 'quantity': '1.5...",https://www.allrecipes.com/recipe/275349/lemon...,rec443,0.340483


In [35]:
result.entries  # Accessing the name of the first recipe

[ResultEntry(id='rec136', fields={'Name': 'Two-Ingredient Pizza Dough', 'Ingredient_Names_Text': 'self-rising_flour greek_yogurt spray', 'Instructions': 'Gather all ingredients. Preheat the oven to 500 degrees F (260 degrees C). Mix flour and Greek yogurt together in a bowl to form a dough  transfer to a work surface floured with self-rising flour. Knead dough, adding more flour as needed to keep dough from being too sticky, about 8 to 10 minutes. Spray a 12-inch pizza pan with cooking spray and spread dough to the edges of the pan. Bake pizza crust in preheated oven for 5 to 7 minutes  add your favorite toppings and return to the preheated oven. Bake until crust is lightly browned, 5 to 7 minutes more. Enjoy!', 'Rating_Value': 4.6, 'Preparation_Time': 15.0, 'Cooking_Time': 10.0, 'Category': ['Dinner'], 'Cuisine': ['Italian'], 'Calories': 116.0, 'Carbohydrates': 18.0, 'Cholesterol': 6.0, 'Fiber': 1.0, 'Protein': 4.0, 'Saturated_Fat': 1.0, 'Sodium': 314.0, 'Sugar': 1.0, 'Fat': 3.0, 'Uns

In [42]:
rows = []

space_names = ["name", "ingredents", "instructions", "rating", "prep_time", "cooking_time", "calories", "protein"]

# order of spaces is the same as during index creation

for entry in result.entries:
    partial_scores = dict(zip(space_names, entry.metadata.partial_scores))
    row = {"id": entry.id, **partial_scores}
    rows.append(row)

df = pd.DataFrame(rows)
df

,id,name,ingredents,instructions,rating,prep_time,cooking_time,calories,protein
0,rec136,0.169071,0.198577,0.125084,0.0,0.0,0.0,0.0,0.0
1,rec443,0.119004,0.109823,0.111656,0.0,0.0,0.0,0.0,0.0
